In [17]:
import pandas as pd
import yfinance as yf
import datetime as dt
from tqdm import tqdm
import time

In [7]:
# Wczytanie danych o spółkach wchodzących w skład S&P 500
sp500_df = pd.read_csv('sp500_ticker_start_end.csv', parse_dates=['start_date', 'end_date'])

In [11]:
# Przechowywanie wyników
top_volatile_all_years = []
bottom_volatile_all_years = []

# Zakres lat
for year in range(2005, 2025):
    print(f"\n\n===== Przetwarzanie roku: {year} =====")
    year_start = dt.datetime(year, 1, 1)
    year_end = dt.datetime(year, 12, 31)

    # Spółki aktywne w danym roku
    in_index = sp500_df[
        (sp500_df['start_date'] <= year_start) &
        ((sp500_df['end_date'].isna()) | (sp500_df['end_date'] >= year_end))
    ]['ticker'].unique().tolist()

    print(f"Liczba spółek w indeksie w roku {year}: {len(in_index)}")

    def download_adjclose_single(ticker, start, end):
        try:
            df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
            if df.empty or 'Adj Close' not in df.columns:
                return None
            adj_close = df[['Adj Close']].copy()
            adj_close.columns = [ticker]
            return adj_close
        except Exception as e:
            print(f"Błąd pobierania dla {ticker}: {e}")
            return None

    all_adj_close = []

    for i, ticker in enumerate(in_index):
        print(f"Pobieranie danych: {i+1}/{len(in_index)} — {ticker}")
        data = download_adjclose_single(ticker, year_start, year_end)
        if data is not None:
            all_adj_close.append(data)
        time.sleep(0.1)  # Pauza dla bezpieczeństwa

    if not all_adj_close:
        print(f"Brak danych dla roku {year}")
        continue

    data = pd.concat(all_adj_close, axis=1)
    returns = data.pct_change().abs()

    # Usuwamy spółki z więcej niż 10% braków danych
    returns = returns.loc[:, returns.isna().mean() < 0.1]

    # Liczymy zmienność jako sumę dziennych absolutnych zmian
    volatility = returns.sum()

    vol_df = pd.DataFrame({
        'ticker': volatility.index,
        'annual_volatility': volatility.values,
        'year': year
    })

    # Dodajemy do zbiorczych list
    top_volatile = vol_df.sort_values('annual_volatility', ascending=False).head(50)
    bottom_volatile = vol_df.sort_values('annual_volatility', ascending=True).head(50)

    top_volatile_all_years.append(top_volatile)
    bottom_volatile_all_years.append(bottom_volatile)

# Łączenie wyników we wspólne tabele
top_combined = pd.concat(top_volatile_all_years).sort_values(['year', 'annual_volatility'], ascending=[True, False])
bottom_combined = pd.concat(bottom_volatile_all_years).sort_values(['year', 'annual_volatility'], ascending=[True, True])



===== Przetwarzanie roku: 2005 =====
Liczba spółek w indeksie w roku 2005: 481
Pobieranie danych: 1/481 — A
Pobieranie danych: 2/481 — AABA



1 Failed download:
['AABA']: YFTzMissingError('possibly delisted; no timezone found')


Pobieranie danych: 3/481 — AAPL
Pobieranie danych: 4/481 — ABC



1 Failed download:
['ABC']: YFTzMissingError('possibly delisted; no timezone found')


Pobieranie danych: 5/481 — ABI


...

Pobieranie danych: 213/484 — HIG
Pobieranie danych: 214/484 — HII
Pobieranie danych: 215/484 — HLT
Pobieranie danych: 216/484 — HOLX
Pobieranie danych: 217/484 — HON
Pobieranie danych: 218/484 — HPE
Pobieranie danych: 219/484 — HPQ
Pobieranie danych: 220/484 — HRL
Pobieranie danych: 221/484 — HSIC
Pobieranie danych: 222/484 — HST
Pobieranie danych: 223/484 — HSY
Pobieranie danych: 224/484 — HUBB
Pobieranie danych: 225/484 — HUM
Pobieranie danych: 226/484 — HWM
Pobieranie danych: 227/484 — IBM
Pobieranie danych: 228/484 — ICE
Pobieranie danych: 229/484 — IDXX
Pobieranie danych: 230/484 — IEX
Pobieranie danych: 231/484 — IFF
Pobieranie danych: 232/484 — INCY
Pobieranie danych: 233/484 — INTC
Pobieranie danych: 234/484 — INTU
Pobieranie danych: 235/484 — INVH
Pobieranie danych: 236/484 — IP
Pobieranie danych: 237/484 — IPG
Pobieranie danych: 238/484 — IQV
Pobieranie danych: 239/484 — IR
Pobieranie danych: 240/484 — IRM
Pobieranie danych: 241/484 — ISRG
Pobieranie danych: 242/484 — IT
Pobi


1 Failed download:
['JNPR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 00:00:00 -> 2024-12-31 00:00:00) (Yahoo error = "No data found, symbol may be delisted")')


Pobieranie danych: 252/484 — JPM
Pobieranie danych: 253/484 — K
Pobieranie danych: 254/484 — KDP
Pobieranie danych: 255/484 — KEY
Pobieranie danych: 256/484 — KEYS
Pobieranie danych: 257/484 — KHC
Pobieranie danych: 258/484 — KIM
Pobieranie danych: 259/484 — KLAC
Pobieranie danych: 260/484 — KMB
Pobieranie danych: 261/484 — KMI
Pobieranie danych: 262/484 — KMX
Pobieranie danych: 263/484 — KO
Pobieranie danych: 264/484 — KR
Pobieranie danych: 265/484 — KVUE
Pobieranie danych: 266/484 — L
Pobieranie danych: 267/484 — LDOS
Pobieranie danych: 268/484 — LEN
Pobieranie danych: 269/484 — LH
Pobieranie danych: 270/484 — LHX
Pobieranie danych: 271/484 — LIN
Pobieranie danych: 272/484 — LKQ
Pobieranie danych: 273/484 — LLY
Pobieranie danych: 274/484 — LMT
Pobieranie danych: 275/484 — LNT
Pobieranie danych: 276/484 — LOW
Pobieranie danych: 277/484 — LRCX
Pobieranie danych: 278/484 — LULU
Pobieranie danych: 279/484 — LUV
Pobieranie danych: 280/484 — LVS
Pobieranie danych: 281/484 — LW
Pobieranie d


1 Failed download:
['PARA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-01-01 00:00:00 -> 2024-12-31 00:00:00) (Yahoo error = "No data found, symbol may be delisted")')


Pobieranie danych: 350/484 — PAYC
Pobieranie danych: 351/484 — PAYX
Pobieranie danych: 352/484 — PCAR
Pobieranie danych: 353/484 — PCG
Pobieranie danych: 354/484 — PEG
Pobieranie danych: 355/484 — PEP
Pobieranie danych: 356/484 — PFE
Pobieranie danych: 357/484 — PFG
Pobieranie danych: 358/484 — PG
Pobieranie danych: 359/484 — PGR
Pobieranie danych: 360/484 — PH
Pobieranie danych: 361/484 — PHM
Pobieranie danych: 362/484 — PKG
Pobieranie danych: 363/484 — PLD
Pobieranie danych: 364/484 — PM
Pobieranie danych: 365/484 — PNC
Pobieranie danych: 366/484 — PNR
Pobieranie danych: 367/484 — PNW
Pobieranie danych: 368/484 — PODD
Pobieranie danych: 369/484 — POOL
Pobieranie danych: 370/484 — PPG
Pobieranie danych: 371/484 — PPL
Pobieranie danych: 372/484 — PRU
Pobieranie danych: 373/484 — PSA
Pobieranie danych: 374/484 — PSX
Pobieranie danych: 375/484 — PTC
Pobieranie danych: 376/484 — PWR
Pobieranie danych: 377/484 — PYPL
Pobieranie danych: 378/484 — QCOM
Pobieranie danych: 379/484 — RCL
Pobier

In [13]:
print("\n===== Top 50 najbardziej zmiennych spółek w każdym roku =====")
print(top_combined.head(10))

print("\n===== Top 50 najmniej zmiennych spółek w każdym roku =====")
print(bottom_combined.head(10))

# zapis do pliku CSV
top_combined.to_csv("Projekty/Projekt nr 3/top_50_volatile_per_year.csv", index=False)
bottom_combined.to_csv("Projekty/Projekt nr 3/bottom_50_volatile_per_year.csv", index=False)


===== Top 50 najbardziej zmiennych spółek w każdym roku =====
    ticker  annual_volatility  year
53     CBE        3730.328937  2005
45     BOL          61.540527  2005
116    GDW          16.929846  2005
122     GR          13.452407  2005
100     EP          12.588366  2005
280    UVN          10.560941  2005
156    KMG           7.486144  2005
248    SOV           7.302069  2005
234    ROH           6.838118  2005
58    CIEN           6.302248  2005

===== Top 50 najmniej zmiennych spółek w każdym roku =====
    ticker  annual_volatility  year
75     CVG           1.146330  2005
151      K           1.399754  2005
215    PEP           1.414600  2005
157     KO           1.454270  2005
47       C           1.467597  2005
289    WFC           1.532172  2005
131    HNZ           1.536338  2005
72     CPB           1.538516  2005
32     BAC           1.556532  2005
257      T           1.562964  2005


In [19]:
# Wczytanie danych
df = pd.read_csv('Projekty/Projekt nr 3/top_50_volatile_per_year.csv')

# Inicjalizacja kolumn
df['1Y_return'] = None
df['1Y_monthly_investment_return'] = None
df['price_source'] = 'Adj Close'

# Funkcja do pobrania danych i obliczenia stóp zwrotu
def compute_returns_for_row(row):
    ticker = row['ticker']
    year = int(row['year'])
    start = dt.datetime(year, 1, 1)
    end = dt.datetime(year, 12, 31)

    try:
        # Pobieranie danych z Yahoo Finance
        df_yf = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
        if df_yf.empty or 'Adj Close' not in df_yf.columns:
            raise ValueError("Brak danych lub brak kolumny 'Adj Close'")

        price_series = df_yf['Adj Close'].dropna()
        if price_series.empty:
            raise ValueError("Brak danych w serii cenowej")

        # Roczna stopa zwrotu (pierwszy i ostatni dzień notowań w roku)
        start_price = float(price_series.iloc[0])
        end_price = float(price_series.iloc[-1])
        return_1y = (end_price - start_price) / start_price

        # Inwestycje miesięczne: pierwszy dostępny dzień >= 1. dnia roboczego każdego miesiąca
        monthly_dates = pd.date_range(start=start, end=end, freq='BMS')
        monthly_prices = []
        for date in monthly_dates:
            possible_dates = price_series.loc[date:].index
            if len(possible_dates) == 0:
                continue
            actual_date = possible_dates[0]
            price = float(price_series.loc[actual_date])
            monthly_prices.append(price)

        if len(monthly_prices) < 2:
            raise ValueError("Za mało danych miesięcznych do obliczenia stopy zwrotu")

        # Comiesięczna inwestycja po 1 jednostce waluty
        shares_bought = [1.0 / price for price in monthly_prices]
        total_units = sum(shares_bought)
        total_cost = len(shares_bought)
        final_price = monthly_prices[-1]
        final_value = total_units * final_price
        monthly_return = (final_value - total_cost) / total_cost

        # Upewniamy się, że zwracamy czyste typy float
        return float(round(return_1y, 6)), float(round(monthly_return, 6))

    except Exception as e:
        print(f"Błąd dla {ticker} w {year}: {e}")
        return float('nan'), float('nan')

# Uruchomienie dla wszystkich wierszy
tqdm.pandas()
df[['1Y_return', '1Y_monthly_investment_return']] = df.progress_apply(
    compute_returns_for_row, axis=1, result_type='expand'
)

# Zapisujemy wynik
df.to_csv('Projekty/Projekt nr 3/top_50_volatile_with_returns.csv', index=False)
print("Plik zapisany jako: top_50_volatile_with_returns.csv")

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]C:\Users\lukas\AppData\Local\Temp\ipykernel_25340\2097849124.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  start_price = float(price_series.iloc[0])
C:\Users\lukas\AppData\Local\Temp\ipykernel_25340\2097849124.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  end_price = float(price_series.iloc[-1])
C:\Users\lukas\AppData\Local\Temp\ipykernel_25340\2097849124.py:39: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(price_series.loc[actual_date])
  0%|▏                                                                                | 2/1000 [00:00<04:46,  3.48

KeyboardInterrupt: 

In [21]:
# Wczytanie danych z pliku
df = pd.read_csv('bottom_50_volatile_per_year.csv')

# Inicjalizacja kolumn
df['1Y_return'] = None
df['1Y_monthly_investment_return'] = None
df['price_source'] = 'Adj Close'

# Główna funkcja obliczająca stopy zwrotu
def compute_returns_for_row(row):
    ticker = row['ticker']
    year = int(row['year'])
    start = dt.datetime(year, 1, 1)
    end = dt.datetime(year, 12, 31)

    try:
        df_yf = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
        if df_yf.empty or 'Adj Close' not in df_yf.columns:
            raise ValueError("Brak danych lub brak kolumny 'Adj Close'")

        price_series = df_yf['Adj Close'].dropna()
        if price_series.empty:
            raise ValueError("Brak danych w serii cenowej")

        # Roczna stopa zwrotu
        start_price = float(price_series.iloc[0])
        end_price = float(price_series.iloc[-1])
        return_1y = (end_price - start_price) / start_price

        # Miesięczne daty inwestycji: 1. dzień roboczy każdego miesiąca
        monthly_dates = pd.date_range(start=start, end=end, freq='BMS')
        monthly_prices = []

        for date in monthly_dates:
            possible_dates = price_series.loc[date:].index
            if len(possible_dates) == 0:
                continue
            actual_date = possible_dates[0]
            price = float(price_series.loc[actual_date])
            monthly_prices.append(price)

        if len(monthly_prices) < 2:
            raise ValueError("Za mało danych miesięcznych do obliczenia stopy zwrotu")

        # Symulacja comiesięcznych inwestycji
        shares_bought = [1.0 / price for price in monthly_prices]
        total_units = sum(shares_bought)
        total_cost = len(shares_bought)
        final_price = monthly_prices[-1]
        final_value = total_units * final_price
        monthly_return = (final_value - total_cost) / total_cost

        return float(round(return_1y, 6)), float(round(monthly_return, 6))

    except Exception as e:
        print(f"Błąd dla {ticker} w {year}: {e}")
        return float('nan'), float('nan')

# Uruchomienie dla wszystkich wierszy
tqdm.pandas()
df[['1Y_return', '1Y_monthly_investment_return']] = df.progress_apply(
    compute_returns_for_row, axis=1, result_type='expand'
)

# Zapisujemy wynik
df.to_csv('bottom_50_volatile_with_returns_final.csv', index=False)
print("Wyniki zapisane do: bottom_50_volatile_with_returns_final.csv")

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]C:\Users\lukas\AppData\Local\Temp\ipykernel_25340\2386594316.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  start_price = float(price_series.iloc[0])
C:\Users\lukas\AppData\Local\Temp\ipykernel_25340\2386594316.py:27: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  end_price = float(price_series.iloc[-1])
C:\Users\lukas\AppData\Local\Temp\ipykernel_25340\2386594316.py:39: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  price = float(price_series.loc[actual_date])
  0%|▏                                                                                | 2/1000 [00:00<03:11,  5.20

KeyboardInterrupt: 